In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
def CNN(X, run): #입력인자 2개의 함수 CNN모델,  
    #filtering 중 comvolusion->filter의 size와 stride해줘! )size= [3,3], 
    #stride는 default값이 1로 잡힌다.
    L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu) 
    #3장으로 1장의 feature map을 만들 수 있는데 총 32개를 만들겠다고 선언, 
    #하나의 convolusion(채널)을 통과했는데 총 32장의 레이어가 만들어짐
    #복수개의 feature맵을 만들어놓고 다시 갈아서 32장의 feature맵을 
    #32개의 filter를 이용해 32개의 채널
    L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2], padding='same') 
    #maxpooling은 이전 L1 레이어를 받아 동일한 깊이에서 pooling작업 sized와 stride만 지정, 
    #'same'은 이전 층과 사이즈 동일하게 하겠다는 말,  
    #padding=’VALID는 적용 안하고 필터 출력해 출력값 작아짐.
    L1 = tf.layers.dropout(L1, 0.7, run)   
    #dropout은 안해도 무방, 근데 규제 주기 위해 자주쓰기는한다. L2norm(많이씀), 
    #L1norm(잘안씀)도 쓰인다.-->규제 주겠다는 말, overfitting 막겠다는 말

    L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu) 
    #64개의 필터로 3x3 크기로 하겠다는 말.
    L2 = tf.layers.max_pooling2d(L2, [2, 2], 2) #패딩 써도 되기는 하는데 그냥 여러 방식 보여줌, 
    #padding=valid는 명시는 하고싶은데 padding 안쓰고 싶을때 
    L2 = tf.layers.dropout(L2, 0.7, run) #dropout을 0.7이라고 미리 줘도 test셋에서 지정되진 않는다. 
    #is_run으로 true, false 있으니 스위치 누를 때 op나 acc에서 지정하면 됨.

    L3 = tf.layers.conv2d(L2, 256, [3, 3], activation=tf.nn.relu)
    L3 = tf.layers.max_pooling2d(L3, [2, 2], [2, 2])
    L3 = tf.layers.dropout(L3, 0.5, run)

    L4 = tf.contrib.layers.flatten(L3)
    L4 = tf.layers.dense(L4, 128, activation=tf.nn.relu) 
    #dense : hiddenlayer의 node수만 지정.
    L4 = tf.layers.dropout(L4, 0.5, run)
    
    hypothesis = tf.layers.dense(L4, 10, activation=None) 
    #기본적으로  none 주면 softmax 해준다.

    return hypothesis 
    #리턴값은 hypo, 예측값을 CNN구조로 짜놓음 hypo=[0.1,0.01,`````0.98]

In [4]:
#mminst로 데이타 불러옴
mnist = input_data.read_data_sets("Data/mnist/", one_hot=True)

W0709 21:07:42.706207  4288 deprecation.py:323] From <ipython-input-4-a97715622e64>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0709 21:07:42.716207  4288 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0709 21:07:42.720207  4288 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to impl

Extracting Data/mnist/train-images-idx3-ubyte.gz


W0709 21:07:43.039226  4288 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0709 21:07:43.055226  4288 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0709 21:07:43.122230  4288 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating

Extracting Data/mnist/train-labels-idx1-ubyte.gz
Extracting Data/mnist/t10k-images-idx3-ubyte.gz
Extracting Data/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1]) #다차원이기 때문에 보통 placeholder는 
#none값이 default지만 얘는 그럼 오류남, 그래서 뒤에 몇개 더 잡아둠
Y = tf.placeholder("float", [None, 10]) #위랑 다르게 얘처럼 문자열로만 잡아둬도됨
run = tf.placeholder(tf.bool) #run은 bool로 잡아둔다

#### conv2d에 관하여
- inputs, outputs size, kernel_size, activation function만 넣어주면 된다.(+ stride, padding)
- xavier_initializer 등 컨볼루션 신경망을 만들기 위한 나머지 수치들은 알아서 결정한다.

In [7]:
hypothesis = CNN(X, run)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [7]:
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

init = tf.global_variables_initializer()

In [9]:
epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

In [10]:
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
    avg_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)  
        #reshape(data가 ) 안해주면 error난다. -1은 none값, 
        #데이터 갯수만큼 잡아줘 앞에 받아줘(100) 뒤에는 못받아주기 때문에 따로 지정해줘야 한다.
        #->데이터 갯수만큼의 28x28x1이 된다. 28x28하면 784가 정확히 되니까!
        _, cost_batch = sess.run([optimizer, cost], #리스트로 넣어주면 op와 cost 버튼 동시에 누를 수 있다. 
                                 #근데 사실 op는 리턴값 필요없긴함 그래서 _처럼 칸을 비워줘서 
                                 #op값의 자리를 placeholder 형식과 같이 자리를 지켜줌. cost만 batch값만 담아낼 수 있다.
                               feed_dict={X: batch_xs, #X값 갖다줌
                                          Y: batch_ys, #y값 갖다줌
                                          run: True})  #run값 갖다줌
        avg_cost += cost_batch / total_batch

    print('Epoch:', '%02d' % (epoch + 1),
          'Avg_cost =', '{:.4f}'.format(avg_cost))

print('finished..')

Epoch: 01 Avg_cost = 0.2548
Epoch: 02 Avg_cost = 0.0747
Epoch: 03 Avg_cost = 0.0552
Epoch: 04 Avg_cost = 0.0398
Epoch: 05 Avg_cost = 0.0326
Epoch: 06 Avg_cost = 0.0280
Epoch: 07 Avg_cost = 0.0221
Epoch: 08 Avg_cost = 0.0185
Epoch: 09 Avg_cost = 0.0174
Epoch: 10 Avg_cost = 0.0146
Epoch: 11 Avg_cost = 0.0139
Epoch: 12 Avg_cost = 0.0105
Epoch: 13 Avg_cost = 0.0080
Epoch: 14 Avg_cost = 0.0108
Epoch: 15 Avg_cost = 0.0077
finished..


In [11]:
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32)) #accuracy 버튼
print('정확도:', sess.run(accuracy, 
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   run: False}))

정확도: 0.9882
